In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
import mlflow
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor


In [2]:
from mlflow import sklearn 
from sklearn.datasets import fetch_california_housing	

In [3]:
housing=fetch_california_housing()

In [4]:
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [5]:
df=pd.DataFrame(housing.data,columns=housing.feature_names)
df['Price']=housing.target

In [6]:
display(df.head(5))

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [7]:
from urllib.parse import urlparse

In [8]:
price=df['Price']

In [9]:
df.drop('Price',axis=1)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [10]:
X_train,X_test,y_train,y_test=train_test_split(df,price,test_size=0.25,random_state=45)

In [11]:
model=RandomForestRegressor()
model.fit(X_train,y_train)
predict=model.predict(X_test)

In [12]:
display(predict)

array([0.65802, 2.28404, 2.41083, ..., 1.05803, 2.43807, 5.00001])

In [13]:
accuracy=accuracy_score(y_true=np.round(y_test),y_pred=np.round(predict))

In [14]:
display(accuracy)

0.998062015503876

In [15]:
## hyperparameter tuning using gridsearchcv 
def hyperparamer_tuning(x_train,Y_train,param_grid):
    rd=RandomForestRegressor()
    grid_search=GridSearchCV(estimator=rd,param_grid=param_grid,cv=5,n_jobs=-1,verbose=2,scoring="neg_mean_squared_error")
    grid_search.fit(x_train,Y_train)
    return grid_search

In [16]:
from mlflow.models import infer_signature
signature=infer_signature(X_train,y_train)



# hyperparameter tuning
param1={
    'n_estimators':[100,200],
    'max_depth':[5,10,None],
    'min_samples_split':[2,5],
    'min_samples_leaf':[1,2]
}

In [17]:
# start mlflow experiment
mlflow.set_registry_uri(uri="http://127.0.0.1:5000")

In [18]:
mlflow.set_experiment("California House Prediction")

2026/01/20 07:38:59 INFO mlflow.tracking.fluent: Experiment with name 'California House Prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///home/test-14900/Projects/Programming/NLP-and-ML-OPS/Projects/House%20Prediction%20Project/mlruns/299686904678475970', creation_time=1768873139260, experiment_id='299686904678475970', last_update_time=1768873139260, lifecycle_stage='active', name='California House Prediction', tags={}>

In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
with mlflow.start_run():
    # perform hyperparam tuning
    grid_search=hyperparamer_tuning(x_train=X_train,Y_train=y_train,param_grid=param1)
    
				# get the best model
    best_model=grid_search.best_estimator_
    
				# evaluate the best model
    y_pred=best_model.predict(X_test)
    mse=mean_squared_error(y_test,y_pred)
    mlflow.log_param("best params n_estimators: ",grid_search.best_params_['n_estimators'])
    mlflow.log_param("best params max_depth: ",grid_search.best_params_['max_depth'])
    mlflow.log_param("best params min_samples_split: ",grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best params min_samples_leaf: ",grid_search.best_params_['min_samples_leaf'])
    mlflow.log_metric("mse:  ",mse)

				# tracking uri
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme
    if tracking_url_type_store!='file':
         mlflow.sklearn.log_model(best_model,"model",registered_model_name="Best randomforest model")
    else:
        mlflow.sklearn.log_model(best_model,"model",signature=signature)
						

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   5.3s
[CV] END max_depth=5, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   5.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   6.0s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.4s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   5.3s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   5.6s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   5.7s
[CV] END max_depth=5, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   6.0s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   6.9s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   7.0s


2026/01/20 07:40:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best randomforest model'.
2026/01/20 07:40:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best randomforest model, version 1
Created version '1' of model 'Best randomforest model'.
2026/01/20 07:40:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run delightful-deer-664 at: http://127.0.0.1:5000/#/experiments/299686904678475970/runs/5e2723ace41d4e03a600b5dfeb55c0e8.
2026/01/20 07:40:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/299686904678475970.
